In [1]:
from excellib import set_excel_color_monthly
from stocklib import get_stock_price, get_OTC_price
from finance import get_yearly, get_df_monthly, get_df_monthly_check, get_df_quarterly
from finance import get_this_quarter, get_quarter_income, get_quarter_finance, set_growth_eps
from finance import get_df_quarterly_all, get_df_yearly_all

import pandas as pd
import numpy as np
import time
from datetime import datetime, timedelta
from tqdm import tqdm

root = 'D:/Mia/Stock/'
path_quarterly = root + 'quarterly/'
path_monthly = root + 'monthly/'
path_month_finance = root + 'monthly_finance/'
path_yearly = root + 'yearly/'
path_finance = root + 'finance/'
folder_name = ''
s_today = datetime.now().strftime('%Y_%m_%d')
print(s_today)

# 公司發行股數
path_company_stock_num = root + 'company_list.xlsx'
path_check = path_quarterly + folder_name +'finance_quarterly_{}.xlsx'.format(s_today)

2022_08_31


In [2]:
# # 取得今日股價
today = datetime.today().date() #- timedelta(days=2)
this_year = today.year
this_quarter = get_this_quarter()

n = 1
if today.weekday() == 0:
    n = 3

print('今天:', today, '第', this_quarter, '季')
# 台灣證券交易所
df_today = get_stock_price(today)
df_today.head(2)

今天: 2022-08-31 第 3 季
取得今日上市股票資訊: 20220831


,name,amount,open,high,low,close
id,,,,,,
0050,元大台灣50,7324,115.95,117.6,115.8,117.6
0051,元大中型100,11,55.10,55.4,55.0,55.4


In [3]:
# # 月報
# #### 彙總報表 (單位：千元)
# https://mops.twse.com.tw/mops/web/t21sc04_ifrs

# # 從月報 篩選出公司
# 上個月營收 > 30億
# 上個月營收成長 > 20% or 去年同季營收成長 > 20%
df_monthly_check = get_df_monthly_check(path_month_finance, income=10, income_growth_MoM=10, income_growth_YoY=10, pre_month=False)
df_monthly_check.head(2)

目前月報年份： 111/7
從月報 篩選出幾家公司
上個月營收 > 10億 上個月營收成長 > 10% or 去年同季營收成長 > 10%
(6777, 14)
所有公司筆數： 971
本月觀察公司筆數： 165


,出表日期,資料年月,公司代號,公司名稱,產業別,營業收入-當月營收,營業收入-上月營收,營業收入-去年當月營收,營業收入-上月比較增減(%),營業收入-去年同月增減(%),累計營業收入-當月累計營收,累計營業收入-去年累計營收,累計營業收入-前期比較增減(%),備註
0,111/08/30,111/7,1101,台泥,水泥工業,101.02,91.46,83.20,10.46,21.42,582.47,583.43,-0.16,"以上營收不包含停業部門營收:111年7月 0 仟元暨110年7月1,366,138仟元；11..."
0,111/08/30,111/6,1101,台泥,水泥工業,91.46,77.34,84.81,18.26,7.84,481.45,500.23,-3.76,"以上營收不包含停業部門營收:111年6月 0 仟元暨110年6月1,162,600仟元；11..."


In [4]:
lst_monthly_id = list(df_monthly_check['公司代號'].unique())

# ==============================
#  取得觀察中公司的季報、預測EPS
# ==============================
df_quarterly_all = get_df_quarterly_all()
df_yearly_all = get_df_yearly_all() # 寫入歷年EPS、本益比，估算今年股價min, max範圍
df_estimate = pd.DataFrame(columns = list(df_quarterly_all.columns)+['普通股數'])
# 取得公司發行股數
df_company_stock_num = pd.read_excel(path_company_stock_num, engine='openpyxl', dtype={'stock_id':str})
df_company_stock_num.set_index('stock_id', inplace=True)

stock_num = len(lst_monthly_id)
print('觀察中公司有', stock_num, '筆')
for i in tqdm(range(stock_num)):
    stock_id = lst_monthly_id[i]
    df = get_df_quarterly(stock_id)
    t_wait = np.random.randint(3,10)
    print(f"{i+1}/{stock_num}  {stock_id}, wait, {t_wait}")
    if df is None:
        print('df is None')
        continue
    if df.loc[df.index[0], '毛利率(%)'] < 20:
        print(stock_id, '：毛利率', df.loc[df.index[0], '毛利率(%)'], '< 20%，跳過')
        continue
    elif df.loc[df.index[0], '營益率(%)'] < 8:
        print(stock_id, '：營益率 < 8%，跳過')
        continue

    # 計算歷年同季的營收成長率、EPS
    df = set_growth_eps(df, df_today, stock_id)

#     將年度EPS寫到歷年excel裡
    lst_year = set(df.index.to_series().apply(lambda x:x[:4]))
    min_year = int(min(lst_year)) # 若最小年是2018 ，只能確定2019都有資料
    lst_year = [y for y in lst_year if int(y) > min_year]
    lst_EPS = [{'year':y, 'eps': df.loc[str(y)+'Q1', 'EPS(Y)']} for y in lst_year]

    df.reset_index(inplace=True)
    df = df.rename(columns={'index':'quarter'}) # reset_index後column名稱是index

    stock_name = ""
    if stock_id in df_today.index:
        stock_name = df_today.loc[stock_id, 'name']
        df_yearly = get_yearly(stock_id, lst_EPS, df['quarter'][0][:4], stock_name)    
        df_yearly_all = pd.concat([df_yearly_all, df_yearly])# df_yearly_all.append(df_yearly, ignore_index=True)

    df.insert(0, 'name', value=stock_name)
    df.insert(0, 'stock_id', value=stock_id)

    df_quarterly_all = pd.concat([df_quarterly_all, df])# df_quarterly_all.append(df, ignore_index=True)
    time.sleep(t_wait)

    # ===============================
    # 預測EPS ( check_EPS )
    # ===============================
    df_new_quarter = get_quarter_finance(stock_id, df, this_year, this_quarter, df_company_stock_num, df_today)
    df_concat = pd.concat([df, df_new_quarter])
    df_concat.sort_values('quarter', ascending=False, inplace=True)
    df_estimate = pd.concat([df_estimate, df_concat])# df_estimate.append(df_concat)

# # 季報
# print('本月觀察公司筆數：', len(df_quarterly_all['stock_id'].unique()), df_quarterly_all['stock_id'].unique())
# set_excel_color_monthly(df_quarterly_all, path_quarterly + folder_name + 'finance_quarterly_{}.xlsx'.format(s_today))

# # 每年 EPS 預估股價
set_excel_color_monthly(df_yearly_all, path_yearly + folder_name + 'finance_yearly_{}.xlsx'.format(s_today))

# 預測EPS ( check_EPS )
df_estimate.to_excel(path_check.replace('finance_quarterly', 'estimate'), index=False)
print(df_estimate.head(2))


# ===============================
# # 取得公司歷年發放股利
# lst = ['2615']
# get_company_dividen(lst)
# ===============================

觀察中公司有 165 筆


  0%|                                                                                          | 0/165 [00:00<?, ?it/s]C:\Users\Mia\anaconda3\envs\stock\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
  1%|▍                                                                               | 1/165 [00:27<1:16:09, 27.87s/it]

1/165  1101, wait, 4
1101 ：毛利率 0.4 < 20%，跳過


C:\Users\Mia\anaconda3\envs\stock\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
  1%|▉                                                                               | 2/165 [00:58<1:19:52, 29.40s/it]

2/165  1102, wait, 8
1102 ：毛利率 17.0 < 20%，跳過


...

164/165  9939, wait, 7
9939  wait  6
'2022'


 99%|███████████████████████████████████████████████████████████████████████████████▌| 164/165 [23:45<00:09,  9.01s/it]C:\Users\Mia\anaconda3\envs\stock\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


165/165  9941, wait, 9
9941  wait  7
'2022'


100%|████████████████████████████████████████████████████████████████████████████████| 165/165 [24:03<00:00,  8.75s/it]


  stock_id name quarter   營業收入   營業成本   營業毛利 營業毛利(毛損)淨額  營業費用  營業利益  稅前淨利  \
0     1319   東陽  2022Q3  55.77    NaN  11.28        NaN  7.62  3.66  5.91   
0     1319   東陽  2022Q2  54.19  41.61  12.58      12.58  7.83  4.75  9.43   

   ... 殖利率(%)  2020 2019 2018 2017 2016 2015         普通股數    year  2021  
0  ...   3.79   NaN  NaN  NaN  NaN  NaN  NaN  591477000.0     NaN   NaN  
0  ...    1.6  64.0  2.6 -6.7 -9.1  NaN  NaN          NaN  2022.0  21.6  

[2 rows x 35 columns]


In [45]:
len(df_estimate['stock_id'].unique())

40

In [6]:
# lst_monthly_id = list(df_monthly_check['公司代號'].unique())
len(lst_monthly_id)

165

In [7]:
sep = 55
lst1 = lst_monthly_id[:sep]
lst2 = lst_monthly_id[sep:sep*2]
lst3 = lst_monthly_id[sep*2:]
print(len(lst1))
print(len(lst2))

55
55


# 取得公司歷年發放股利

In [10]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from datetime import date 
from tqdm import tqdm

path_dividen = root + 'company_dividen.xlsx'
# df_dividen_all = pd.DataFrame(columns=['stock_id', 'year', 'cash', 'stock'])
df_dividen_all = pd.read_excel(path_dividen, engine='openpyxl', dtype={'stock_id':str})

ua = UserAgent()
agent = ua.random

for stock_id in tqdm(lst3):
     #or not stock_id in lst_company:
    df_exist = df_dividen_all[df_dividen_all['stock_id']==stock_id]
    if len(df_exist) > 0:
        if len(df_exist[df_exist['year']==this_year]) > 0:
            continue
#     stock_id = lst_company_1[i]
    t_wait = np.random.randint(15, 25)
    print(stock_id, 'wait:', t_wait)
    url = 'https://goodinfo.tw/StockInfo/StockDetail.asp?STOCK_ID={}'.format(stock_id)

    headers = {'user-agent':agent}
    res = requests.get(url, headers=headers)
    res.encoding = 'utf-8'
    soup = BeautifulSoup(res.text, 'lxml')

    tb = soup.find(id='FINANCE_DIVIDEND')
    
    try:
        df_dividen = pd.read_html(str(tb))[0]
    except:
        print(stock_id, 'Error No tables found')
        continue
    df_dividen = df_dividen.iloc[:,:-1]
    df_dividen.columns = ['year', 'cash', 'stock']
    df_dividen.drop(df_dividen.index[:3], inplace=True)
    df_dividen.drop(df_dividen.index[-1], inplace=True)
    df_dividen['year'] = df_dividen['year'].astype(int)
    df_dividen = df_dividen[df_dividen['year']>2014]
    if len(df_exist) > 0:
        df_dividen = df_dividen[~df_dividen['year'].isin(df_exist['year'])]
        print('Add this year', this_year)
        print(df_dividen)
    df_dividen.insert(0, 'stock_id', value=stock_id)
    df_dividen_all = pd.concat([df_dividen_all, df_dividen]) # df_dividen_all.append(df_dividen)
    time.sleep(t_wait)
    
df_dividen_all.to_excel(path_dividen, index=False)

# 合併發行股數 與 歷年股利Excel
path_company = root + 'company_list.xlsx'
path_dividen = root + 'company_dividen.xlsx'
path_company_info = root + 'company_info.xlsx'
import shutil
shutil.copy2(path_company_info, path_company_info.replace('.xlsx', '_backup.xlsx'))

df_1 = pd.read_excel(path_company, engine='openpyxl')
df_2 = pd.read_excel(path_dividen, engine='openpyxl')
print(df_1.shape)
print(df_2.shape)
df_all = df_1.set_index('stock_id').join(df_2.set_index('stock_id'))
print(df_all.shape)
df_all.head(2)
df_all.reset_index(inplace=True)
df_all.to_excel(path_company_info, index=False)

  0%|                                                                                           | 0/55 [00:00<?, ?it/s]

3706 wait: 19
Add this year 2022
   year cash stock
3  2022    2     0


  2%|█▌                                                                                 | 1/55 [00:22<20:30, 22.79s/it]

3711 wait: 21
Add this year 2022
   year cash stock
3  2022    7     0


  4%|███                                                                                | 2/55 [00:45<20:17, 22.98s/it]

...

9939 wait: 16
Add this year 2022
   year cash stock
3  2022  4.3     0


 98%|████████████████████████████████████████████████████████████████████████████████▌ | 54/55 [14:09<00:15, 15.62s/it]

9941 wait: 16
Add this year 2022
   year cash stock
3  2022  5.5   1.7


100%|██████████████████████████████████████████████████████████████████████████████████| 55/55 [14:27<00:00, 15.77s/it]


(934, 3)
(4201, 4)
(4500, 5)
